## 读写文件
有时我们希望保
存训练的模型，以备将来在各种环境中使⽤（⽐如在部署中进⾏预测）。此外，当运⾏⼀个耗时较⻓的训练过
程时，最佳的做法是定期保存中间结果，以确保在服务器电源被不⼩⼼断掉时，我们不会损失⼏天的计算结
果。因此，现在是时候学习如何加载和存储权重向量和整个模型了。


### 加载和保存张量
对于单个张量，我们可以直接调⽤load和save函数分别读写它们。这两个函数都要求我们提供⼀个名称，save要
求将要保存的变量作为输⼊。


In [2]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [3]:
# 可以存储⼀个张量列表，然后把它们读回内存
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [4]:
# 甚⾄可以写⼊或读取从字符串映射到张量的字典。当我们要读取或写⼊模型中的所有权重时，这很⽅便。
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 加载和保存模型参数
保存单个权重向量（或其他张量）确实有⽤，但是如果我们想保存整个模型，并在以后加载它们，单独保存
每个向量则会变得很⿇烦。毕竟，我们可能有数百个参数散布在各处。因此，深度学习框架提供了内置函数
来保存和加载整个⽹络。需要注意的⼀个重要细节是，这将保存模型的参数⽽不是保存整个模型。  

例如，如
果我们有⼀个3层多层感知机，我们需要单独指定架构。因为模型本⾝可以包含任意代码，所以模型本⾝难以
序列化。因此，为了恢复模型，我们需要⽤代码⽣成架构，然后从磁盘加载参数。让我们从熟悉的多层感知
机开始尝试⼀下。

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
print(Y)


tensor([[ 0.0088,  0.0871,  0.3723, -0.0711, -0.1907,  0.0368, -0.1042,  0.0279,
          0.1736,  0.3211],
        [-0.2162,  0.0947,  0.4789,  0.1285,  0.1699,  0.6509, -0.2926, -0.3162,
         -0.0762, -0.2213]], grad_fn=<AddmmBackward0>)


In [7]:
# 接下来，我们将模型的参数存储在⼀个叫做“mlp.params”的⽂件中
torch.save(net.state_dict(), 'mlp.params')


In [8]:
#为了恢复模型，实例化了原始多层感知机模型的⼀个备份。这⾥我们不需要随机初始化模型参数，⽽是直接读取⽂件中存储的参数。
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [9]:
# 验证一下两个实例的计算结果
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结
* save和load函数可⽤于张量对象的⽂件读写
* 我们可以通过参数字典保存和加载⽹络的全部参数
* 保存架构必须在代码中完成，⽽不是在参数中完成